In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch.nn.functional as F

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [64]:
# Charger le modèle GPT-2 LARGE et le tokenizer
model_name = "gpt2-large"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_nam  # Mode évaluation

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3840, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=1280)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=5120, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=5120)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

In [65]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = model.to(device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 0 has a total capacity of 14.57 GiB of which 20.75 MiB is free. Process 1369089 has 14.54 GiB memory in use. Of the allocated memory 14.36 GiB is allocated by PyTorch, and 65.96 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [57]:
# Préparer le prompt
prompt = "The Space Needle is in"
# 'inputs' est un dictionnaire contenant les représentations numériques du prompt
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)


In [58]:
outputs.keys()


odict_keys(['logits', 'past_key_values'])

In [59]:
# Récupérer les logits pour le token suivant (dernier token de la séquence)
logits = outputs.logits  # forme: (batch_size, séquence, vocab_size)
next_token_logits = logits[0, -1, :]

In [60]:
logits.shape # 1 ligne = 1 prompte, 6 colonnes = 6 token, et 50 257 mots de vocabulaires possible (token possible)

torch.Size([1, 6, 50257])

In [61]:
# Conversion des logits en probabilités
probabilities = F.softmax(next_token_logits, dim=0)

In [62]:
# Affichage des 10 tokens les plus probables
top_k = 10
top_probs, top_indices = torch.topk(probabilities, top_k)
top_tokens = [tokenizer.decode([idx]) for idx in top_indices]



In [63]:
print("Top 10 des prédictions:")
for token, prob in zip(top_tokens, top_probs):
    print(f"{token.strip()}: {prob.item():.4f}")

Top 10 des prédictions:
the: 0.2812
Seattle: 0.0986
a: 0.0924
danger: 0.0738
fact: 0.0220
its: 0.0150
need: 0.0132
good: 0.0115
an: 0.0114
full: 0.0099
